# LLM 과 OpenSearch 를 이용해서 RAG 아키텍처를 구축합니다.


여기서는 OpenSearch 가 설치된 것을 가정합니다.

## 0. 필요한 변수 불러오기

In [1]:
# OpenSearch와 통신하기 위해서 앞서 저장했던 변수를 불러옵니다.
%store -r opensearch_user_id opensearch_user_password opensearch_domain_name opensearch_domain_endpoint

print(f"OpenSearch User Id: {opensearch_user_id}")
print(f"OpenSearch User Password: {opensearch_user_password}")
print(f"OpenSearch Domain Name: {opensearch_domain_name}")
print(f"OpenSearch Domain Endpoint: {opensearch_domain_endpoint}")
print()
    
    
# SageMaker Endpoint 와 통신하기 위해서 저장했던 변수를 불러옵니다.
%store -r endpoint_name
print(f"SageMaker EndpointName: {endpoint_name}")
print()

OpenSearch User Id: raguser
OpenSearch User Password: Passw0rd1!
OpenSearch Domain Name: rag-hol-c10dc17d
OpenSearch Domain Endpoint: https://search-rag-hol-c10dc17d-ddlzarx5gvyn65qc2e6og2frfq.us-west-2.es.amazonaws.com

SageMaker EndpointName: llama-3-2-1b-instruct-2024-11-21-01-23-27-964



In [2]:
# 저장이 되어 있지 않다면 직접 관련 정보를 입력합니다.

# opensearch_user_id = 'raguser'
# opensearch_user_password = 'Passw0rd1!'

# opensearch_domain_name = 'rag-hol-6efd6d14'
# opensearch_domain_endpoint = 'https://search-rag-hol-6efd6d14-5wbkv7qjrlci47h5ka63cw5fxy.us-west-2.es.amazonaws.com'

# endpoint_name='llama-3-2-1b-instruct-2024-11-18-14-18-13-972'

## 1. OpenSearchVectorSearch 생성

In [3]:
import boto3
from langchain_aws import BedrockEmbeddings
from langchain_community.vectorstores import OpenSearchVectorSearch

# Embedding Model 생성
bedrock_runtime = boto3.client('bedrock-runtime')
embedding = BedrockEmbeddings(
    client=bedrock_runtime,
    model_id="amazon.titan-embed-text-v2:0", # Titan Text Embeddings V2
)

# Index Name 정의
index_name = "rag-index-name"

# OpenSearch 인증 정보
http_auth = (opensearch_user_id, opensearch_user_password)

# OpenSearchVectorSearch 생성
vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=embedding,
    http_auth=http_auth, # http_auth
)

In [4]:
# from opensearchpy import OpenSearch, RequestsHttpConnection

# # Delete index
# os_client = OpenSearch(
#     region=region,
#     hosts=[{'host': opensearch_domain_endpoint.replace("https://", ""), 'port': 443}],
#     http_auth=http_auth,
#     use_ssl=True,
#     verify_certs=True,
#     connection_class=RequestsHttpConnection
# )
# os_client.indices.delete(index=index_name)

## 4. RAG 구성으로 호출 - Bedrock 사용해서 응답 생성

### 4-1. Knowledge Base 에서 질문과 유사도가 높은 내용 검색

In [5]:
query = "아마존의 비전은 무엇인가요"
k = 3
results=vector_db.similarity_search_with_score(
    query,
    k=k
)
page_contents = [res[0].page_content for res in results]
context = page_contents[0]
print(context)

Setting the bar high in our approach to hiring has been, and will continue to be, thesingle most important element of Amazon.com’s success. It’s not easy to work here (when I interview people I tell them, “Y ou can work long, hard, or smart, but at Amazon.com you can’t choose two out of three”), but we are working to build something important, somethingthat matters to our customers, something that we can all tell our grandchildren about. Such things aren’t meant tobe easy. We are incredibly fortunate to have this group of dedicated employees whose sacrifices and passionbuild Amazon.com. Goals for 1998 We are still in the early stages of learning how to bring new value to our customers through Internet commerce and merchandising. Our goal remains to continue to solidify and extend our brand and customer base.This requires sustained investment in systems and infrastructure to support outstanding customer convenience,selection, and service while we grow.


### 4-2. 유사도가 높은 내용(Context)을 포함한 프롬프트 작성

In [6]:
prompt = f"""
<context> 태그 안의 내용을 참고해서 사용자 질문에 답변해줘.
서두는 건너뛰고 질문에 대한 답변만 해줘

<context>
{context}
</context>

User: {query}
Assistant:
"""

prompt

'\n<context> 태그 안의 내용을 참고해서 사용자 질문에 답변해줘.\n서두는 건너뛰고 질문에 대한 답변만 해줘\n\n<context>\nSetting the bar high in our approach to hiring has been, and will continue to be, thesingle most important element of Amazon.com’s success. It’s not easy to work here (when I interview people I tell them, “Y ou can work long, hard, or smart, but at Amazon.com you can’t choose two out of three”), but we are working to build something important, somethingthat matters to our customers, something that we can all tell our grandchildren about. Such things aren’t meant tobe easy. We are incredibly fortunate to have this group of dedicated employees whose sacrifices and passionbuild Amazon.com. Goals for 1998 We are still in the early stages of learning how to bring new value to our customers through Internet commerce and merchandising. Our goal remains to continue to solidify and extend our brand and customer base.This requires sustained investment in systems and infrastructure to support outstanding customer conv

### 4-2. 응답 생성

In [7]:
import boto3
import json

# Bedrock 클라이언트 생성
bedrock_runtime = boto3.client('bedrock-runtime')

# converse API 호출
response = bedrock_runtime.converse(
    modelId="anthropic.claude-3-haiku-20240307-v1:0",
    messages=[{"role": "user", "content": [{"text": prompt}]}],
    system=[{"text": "당신은 친절한 AI 친구입니다. 상냥하게 답변해주세요."}],
    inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9}
)

# 응답 처리
ai_message = response["output"]["message"]
output_text = ai_message["content"][0]["text"]
print(f"{output_text}")

아마존의 비전은 고객에게 새로운 가치를 제공하는 것입니다. 이를 위해 지속적인 투자를 통해 고객 편의성, 선택권, 서비스 품질을 향상시키면서 브랜드와 고객 기반을 공고히 하고자 합니다.


## 5. RAG 구성으로 호출 - Llama 3.2 사용해서 응답 생성

### 5-1. Knowledge Base 에서 질문과 유사도가 높은 내용 검색

In [8]:
query = "What is Amazon's vision?"
k = 3
results=vector_db.similarity_search_with_score(
    query,
    k=k
)
page_contents = [res[0].page_content for res in results]
context = page_contents[0]
print(context)

We operate in a diverse number of market segments, but what ties Amazon together is our joint missionto make customers’ lives better and easier every day. This is true across every customer segment we serve(consumers, sellers, brands, developers, enterprises, and creators). At our best, we’re not just customerobsessed, but also inventive, thinking several years out, learning like crazy, scrappy, delivering quickly, andoperating like the world’s biggest start-up. We spend enormous energy thinking about how to empower builders, inside and outside of our company. We characterize builders as people who like to invent. They like to dissect a customer experience, assess what’swrong with it, and reinvent it. Builders tend not to be satisfied until the customer experience is perfect.This doesn’t hinder them from delivering improvements along the way, but it drives them to keep tinkeringand iterating continually.


### 5-2. 유사도가 높은 내용(Context)을 포함한 프롬프트 작성

In [9]:
prompt = f"""
Context information is below.
---------------------
{context}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query}
Answer: 
"""
prompt

"\nContext information is below.\n---------------------\nWe operate in a diverse number of market segments, but what ties Amazon together is our joint missionto make customers’ lives better and easier every day. This is true across every customer segment we serve(consumers, sellers, brands, developers, enterprises, and creators). At our best, we’re not just customerobsessed, but also inventive, thinking several years out, learning like crazy, scrappy, delivering quickly, andoperating like the world’s biggest start-up. We spend enormous energy thinking about how to empower builders, inside and outside of our company. We characterize builders as people who like to invent. They like to dissect a customer experience, assess what’swrong with it, and reinvent it. Builders tend not to be satisfied until the customer experience is perfect.This doesn’t hinder them from delivering improvements along the way, but it drives them to keep tinkeringand iterating continually.\n---------------------\nG

In [10]:
import json
import boto3
import datetime

region = boto3.Session().region_name
sagemaker_runtime = boto3.client(
    "sagemaker-runtime",
    region_name=region
)

In [11]:
client = boto3.client("sagemaker-runtime", region_name='us-west-2')

# Payload 직성
payload = {
    "inputs": prompt,
    "parameters": {
        "max_new_tokens": 512,
        "top_p": 0.9,
        "temperature": 0.1,
    }
}

# SageMaer Endpoint 호출
response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    # EndpointName="llama-3-2-1b-instruct-2024-11-18-14-18-13-972",
    ContentType="application/json",
    Body=json.dumps(payload),
)

# 응답 처리
end_time = datetime.datetime.now()
response_body = json.loads(response['Body'].read().decode())


# 결과 출력
print(f"# Response\n- {response_body['generated_text']}\n")

ValidationError: An error occurred (ValidationError) when calling the InvokeEndpoint operation: Endpoint llama-3-2-1b-instruct-2024-11-21-01-23-27-964 of account 134778062208 not found.